# PILLOW

In [15]:
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
from IPython.display import display
import sys
import pandas as pd

# np.set_printoptions(threshold=sys.maxsize)   

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/home/hoangth34/mobilenet_from_c_scratch/model/v3-large-minimalistic_224_1.0_uint8.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
image_path = "sample_data/Snail.jpg"
input_shape = input_details[0]['shape']
image = Image.open(image_path)
image = image.resize((224,224))
input_data_pil = np.expand_dims(np.array(image), 0)
interpreter.set_tensor(input_details[0]['index'], input_data_pil)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])

# Get Labels
with open("mobilenet_labels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]
len(labels)

print("max value of output_data: ", output_data.max())
print("Index of output_data: ", max(output_data).argmax())
print("Label of output_data: ", labels[max(output_data).argmax()])

max value of output_data:  220
Index of output_data:  114
Label of output_data:  snail


# OPENCV

In [16]:
img_cv2 = cv2.imread(image_path)
img_cv2 = cv2.resize(img_cv2, (224,224))

img_cv2_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
input_data_cv2 = np.expand_dims(img_cv2_rgb, 0)
interpreter.set_tensor(input_details[0]['index'], input_data_cv2)

interpreter.invoke()

output_data_cv2 = interpreter.get_tensor(output_details[0]['index'])
print("max value of output_data_cv2: ", output_data_cv2.max())
print("Index of output_data_cv2: ", max(output_data_cv2).argmax())
print("Label of output_data_cv2: ", labels[max(output_data_cv2).argmax()])

max value of output_data_cv2:  239
Index of output_data_cv2:  114
Label of output_data_cv2:  snail


# Analysis

In [23]:
tf.lite.experimental.Analyzer.analyze(model_path="/home/hoangth34/mobilenet_from_c_scratch/model/v3-large-minimalistic_224_1.0_uint8.tflite")

=== /home/hoangth34/mobilenet_from_c_scratch/model/v3-large-minimalistic_224_1.0_uint8.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #159 and tensor #3 and tensor #1 as input and produces tensor #2 as output.

Subgraph#0(T#159) -> [T#16]
  Op#0 CONV_2D(T#159, T#3, T#1[4633, 5035, 13629, 6544, 13061, ...]) -> [T#2]
  Op#1 DEPTHWISE_CONV_2D(T#2, T#20, T#19[19, 6, 20, 20, 16, ...]) -> [T#18]
  Op#2 CONV_2D(T#18, T#23, T#21[-1216, 1775, -2946, 1438, 641, ...]) -> [T#22]
  Op#3 ADD(T#22, T#2) -> [T#17]
  Op#4 CONV_2D(T#17, T#29, T#27[-153, 690, 88, 565, 230, ...]) -> [T#28]
  Op#5 DEPTHWISE_CONV_2D(T#28, T#26, T#25[141, 12, 23, -756, 191, ...]) -> [T#24]
  Op#6 CONV_2D(T#24, T#32, T#30[-2097, -3646, 974, 210, -2096, ...]) -> [T#31]
  Op#7 CONV_2D(T#31, T#87, T#85[-170, -612, 557, 498, -429, ...]) -> [T#86]
  Op#8 DEPTHWISE_CONV_2D(T#86, T#84, T#83[15, 120, 8, 122